# WORD GRAPH PROJECT - step 1

We will use the work done here 

https://www.kaggle.com/datasets/moxxis/harry-potter-lstm?resource=download&select=Harry_Potter_all_books_preprocessed.txt

so thank to this boy to provide all the data we searched and to prepocess even them (this should help a lot). 

We start by reading everything.

In [50]:
HP_file = open("HP_books.txt", "r")
HP_text = HP_file.read()

Now we split everything and count how many words there are. We also replace some character with a space (since we had some problem with some sentences) and we lower everything. We set | as the ending character.

In [51]:
special_char = ['.', ',', '?', '!', 'â€˜', 'â€¢', '~', '(', ')']
for s_char in special_char:
    HP_text = HP_text.replace(s_char, f" {s_char} | ")
HP_text = HP_text.lower()
HP_words = HP_text.split()
len(HP_words)

1259308

And we count the numbers of single words.

In [52]:
HP_set_words = set(HP_words)
len(HP_set_words)

23232

This is something really cool to notice. We now there are approximetely 170.000 words in the english vocabulary and this means that HP really minimize the space of the words (relatively talking).
Now we want to count the occurrencies of the single words. 

In [53]:
HP_occourrencies_dict = {}
for word in HP_words:
    if word == '':
        continue
    if word in HP_occourrencies_dict.keys():
        HP_occourrencies_dict[word] += 1
    else:
        HP_occourrencies_dict[word] = 1
HP_sorted_occourrencies_dict = dict(sorted(HP_occourrencies_dict.items(), key=lambda item: item[1], reverse=True))

In [54]:
k = 0
lim = 10
for unique_word in HP_sorted_occourrencies_dict:
    if k < lim:
        print(f"{unique_word} occourred {HP_sorted_occourrencies_dict[unique_word]}")
        k = k + 1

| occourred 86068
. occourred 67794
the occourred 51921
and occourred 27607
to occourred 26852
of occourred 21843
a occourred 21074
he occourred 20430
harry occourred 16750
was occourred 15644


Let's implement a class for the graph.

In [152]:
import random

class WordGraph:
    def __init__(self, text): # we assume the text is already prepocessed
        self.text = text
        self.words = self.text.split()
        self.unique_word = set(self.words)
        self.words_dict = {u_word : {} for u_word in self.unique_word}
        self.compute_graph()
    
    def compute_graph(self):
        for predecessor, successor in zip(self.words[:-1], self.words[1:]):
            if predecessor == '' or successor == '':
                continue
            if successor not in self.words_dict[predecessor].keys():
                self.words_dict[predecessor][successor] = 1
            else:
                self.words_dict[predecessor][successor] += 1
    
    def print_graph(self):
        for words in self.words_dict:
            print(f"{words} ")
            for word in self.words_dict[words]:
                print(f" |->{word} : {self.words_dict[words][word] }")
            print(" ")
    
    def print_word(self, key_word):
        key_word = key_word.lower()
        print(f"{key_word} ")
        try:
            w_dict = self.words_dict[key_word]
        except:
            print(" |->Word not found")
            return
        for word in w_dict:
            print(f" |->{word} : {self.words_dict[key_word][word] }")
    
    def generate_from_word(self, key_word, max_deep = 15):
        key_word = key_word.lower()
        if max_deep == 0 or key_word == '|':
            print()
            return
        print(key_word, end=' ')
        try:
            w_dict = self.words_dict[key_word]
        except:
            print("Word not found")
            return 
        cumulative_space = {}
        c = 0 
        for word in w_dict:
            c = c + self.words_dict[key_word][word]
            cumulative_space[c] = word
        index = random.randint(0, c)
        for ind in cumulative_space.keys(): # we could optimize this with a binary search. now it's not needed.
            if index < ind:
                self.generate_from_word(cumulative_space[ind], max_deep - 1)
                return 

In [153]:
HP_wg = WordGraph(HP_text)

We implement also a function called "generate from word" that simply, given a word, create a cumulative probabilistic function and generate from it the next word. the cumulative function just use the sum of the occurrencies in order. sampled by it we take the correspective word: if it finds an end character (or goes too deep in the recursion) it terminates. 

In [186]:
HP_wg.generate_from_word('hi', max_deep=25)

hi she said hermione muttered alohomora . 
